In [1]:
import os
import keras
import numpy as np
from glob import glob
from tqdm import tqdm
import tensorflow as tf

import pandas as pd
from keras.preprocessing.image import ImageDataGenerator


import seaborn as sns
import matplotlib.pyplot as plt


from keras import Sequential
from keras.layers import GlobalAvgPool2D, Dense, Dropout
from keras.models import load_model


from tensorflow.keras.applications import ResNet50, ResNet50V2, InceptionV3, Xception
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    validation_split=0.2)

test_gen = ImageDataGenerator(rescale=1./255)

path = "C:/Users/PC/Downloads/coffeebeansdataset/"
train_ds = train_gen.flow_from_directory(path + "train", target_size=(256,256), shuffle=True, batch_size=32, subset="training", class_mode='binary')
valid_ds = train_gen.flow_from_directory(path + "train", target_size=(256,256), shuffle=True, batch_size=32, subset="validation", class_mode='binary')
test_ds = train_gen.flow_from_directory(path + "test", target_size=(256,256), shuffle=True, batch_size=32, class_mode='binary')

Found 960 images belonging to 4 classes.
Found 240 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [6]:
class_names = sorted(os.listdir("C:/Users/PC/Downloads/coffeebeansdataset/train"))
n_classes = len(class_names)
class_names

['Dark', 'Green', 'Light', 'Medium']

In [7]:
name = "ResNet50V2"
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(256,256,3))
base_model.trainable = False

resnet_50V2 = Sequential([
     base_model, 
     GlobalAvgPool2D(),
     Dense(256, activation='relu'),
     Dropout(0.2),
     Dense(n_classes, activation='softmax')
 ])


resnet_50V2.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

cbs = [EarlyStopping(patience=3, restore_best_weights=True), ModelCheckpoint(name + ".h5", save_best_only=True)]

resnet_50V2.fit(train_ds, validation_data=valid_ds, epochs=50, callbacks=cbs)

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.6722 - accuracy: 0.7333

C:\Users\PC\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 55s 2s/step - loss: 0.6722 - accuracy: 0.7333 - val_loss: 0.1693 - val_accuracy: 0.9500
Epoch 2/50
30/30 [==============================] - 48s 2s/step - loss: 0.1509 - accuracy: 0.9479 - val_loss: 0.1053 - val_accuracy: 0.9625
Epoch 3/50
30/30 [==============================] - 46s 2s/step - loss: 0.1063 - accuracy: 0.9604 - val_loss: 0.1307 - val_accuracy: 0.9333
Epoch 4/50
30/30 [==============================] - 46s 2s/step - loss: 0.1099 - accuracy: 0.9604 - val_loss: 0.0672 - val_accuracy: 0.9917
Epoch 5/50
30/30 [==============================] - 47s 2s/step - loss: 0.0688 - accuracy: 0.9781 - val_loss: 0.0771 - val_accuracy: 0.9667
Epoch 6/50
30/30 [==============================] - 53s 2s/step - loss: 0.0616 - accuracy: 0.9823 - val_loss: 0.0546 - val_accuracy: 0.9917
Epoch 7/50
30/30 [==============================] - 48s 2s/step - loss: 0.0401 - accuracy: 0.9865 - val_loss: 0.0687 - val_accuracy: 0.9750
Epoch 8/50
30/30 [=============

In [8]:
model_path = 'ResNet50V2.h5'
model = load_model(model_path)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 24090372 (91.90 MB)
Trainable params: 525572 (2.00 MB)
Non-trainable params: 23564800 (89.89 MB)
___________

In [10]:
model.evaluate(test_ds)

13/13 [==============================] - 18s 1s/step - loss: 0.0157 - accuracy: 0.9975


[0.015661807730793953, 0.9975000023841858]